<a href="https://colab.research.google.com/github/Shivam10816/cs6910_assignment3/blob/main/Assignment3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!pip install wandb


In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [126]:
import wandb
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()


class Lang:
    def __init__(self, name):
        self.name = name
        self.char2index = {'#': 0, '$': 1, '^': 2}
        self.char2count = {'#': 1, '$': 1, '^': 1}
        self.index2char = {0: '#', 1: '$', 2: '^'}
        self.n_chars = 3  # Count
        self.data = {}
        

    def addWord(self, word):
        for char in word:
            self.addChar(char)

    def addChar(self, char):
        if char not in self.char2index:
            self.char2index[char] = self.n_chars
            self.char2count[char] = 1
            self.index2char[self.n_chars] = char
            self.n_chars += 1
        else:
            self.char2count[char] += 1



class EncodedData:
  def __init__(self,lang):
    self.train_df = pd.read_csv(f"drive/MyDrive/aksharantar_sampled/{lang}/{lang}_train.csv", header = None)
    self.val_df = pd.read_csv(f"drive/MyDrive/aksharantar_sampled/{lang}/{lang}_valid.csv", header = None)
    self.test_df = pd.read_csv(f"drive/MyDrive/aksharantar_sampled/{lang}/{lang}_test.csv", header = None)

    self.input_lang = Lang('eng')
    self.output_lang = Lang(lang)

    # add the words to the respective languages
    for i in range(len(self.train_df)):
        
        self.input_lang.addWord(self.train_df[0][i])
        self.output_lang.addWord(self.train_df[1][i])
    
    max_len_1,max_len_2 = self.max_word_lengths(self.train_df)
    self.train_xE,self.train_yE = self.preprocessing(self.train_df,max_len_1,max_len_2)

    max_len_1,max_len_2 = self.max_word_lengths(self.val_df)
    self.val_xE,self.val_yE = self.preprocessing(self.val_df,max_len_1,max_len_2)

    max_len_1,max_len_2 = self.max_word_lengths(self.val_df)
    self.val_xE,self.val_yE = self.preprocessing(self.val_df,max_len_1,max_len_2)

    print(type(self.train_xE))
    
  def max_word_lengths(self,df):
    max_len_1 = df.iloc[:,0].str.len().max()
    max_len_2 = df.iloc[:,1].str.len().max()
    return max_len_1, max_len_2



  def preprocessing(self,df,max_len_1,max_len_2):
    
    M1 = np.zeros((len(df),max_len_1+1))
    M2 = np.zeros((len(df),max_len_2+2))

    #print(M2.shape)
    
    # Encode column 1
    col1 = df.iloc[:, 0].str.lower().str.split()

    for i in range(len(col1)) :
      word = col1[i][0]
      # print(word," " ,word[0],word[1])
      for j in range(len(word)):
        char =word[j]
        if(self.input_lang.char2index.get(char) is not None):
          M1[i][j]=self.input_lang.char2index.get(char)
        else:
          M1[i][j]=1
    # print(col1)
    #encoded_col1 = np.array([encode(words, dict1, max_len_1) for words in col1])
    
    # Encode column 2
    col2 = df.iloc[:, 1].str.lower().str.split()
    # print((col2[0]))
    for i in range(len(col2)) :
      word = col2[i][0]
      # print(word," " ,word[0],word[1])
      M2[i][0]=2
      for j in range(len(word)):
        char =word[j]
        if(self.output_lang.char2index.get(char) is not None):
          M2[i][j+1]=self.output_lang.char2index.get(char)
        else:
          M2[i][j+1]=1
      
    
    
    return torch.from_numpy(M1),torch.from_numpy(M2)


class EncoderRNN(nn.Module):
    def __init__(self, input_size, embedding_size, num_encoders, hidden_size, bidirectional=False, cell='GRU',dropout=0.0):
        super(EncoderRNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_encoders = num_encoders
        self.bidirectional = bidirectional
        self.cell = cell
        self.embedding_size = embedding_size
        self.embedding = nn.Embedding(input_size, embedding_size)
        
        if self.cell == 'RNN':
            self.encoder_rnn = nn.RNN(embedding_size, hidden_size, num_layers=num_encoders,
                                      bidirectional=bidirectional,dropout=dropout )
        elif self.cell == 'LSTM':
            self.encoder_rnn = nn.LSTM(embedding_size, hidden_size, num_layers=num_encoders,
                                       bidirectional=bidirectional,dropout=dropout )
        elif self.cell == 'GRU':
            self.encoder_rnn = nn.GRU(embedding_size, hidden_size, num_layers=num_encoders,
                                      bidirectional=bidirectional, dropout=dropout)
        
    def init_hidden(self, batch_size):
        num_directions = 2 if self.bidirectional else 1
        if self.cell == 'LSTM':
            return (torch.zeros(self.num_encoders * num_directions, batch_size, self.hidden_size, device=device),
                    torch.zeros(self.num_encoders * num_directions, batch_size, self.hidden_size, device=device))
        else:
            return torch.zeros(self.num_encoders * num_directions, batch_size, self.hidden_size, device=device)
        
    def forward(self, input_seq):
        batch_size = input_seq.size(0)
        
        # Embedding layer
        embedded = self.embedding(input_seq.long()).view(-1,batch_size, self.embedding_size)
        
        # Encoder layers
        encoder_hidden = self.init_hidden(batch_size)

        
        if self.cell == 'LSTM':
            encoder_cell = self.init_hidden(batch_size)
            encoder_outputs, (encoder_hidden, encoder_cell) = self.encoder_rnn(embedded, (encoder_hidden, encoder_cell))
        else:
            encoder_outputs, encoder_hidden = self.encoder_rnn(embedded, encoder_hidden)
        if self.bidirectional:
            encoder_hidden = torch.cat((encoder_hidden[-2,:,:], encoder_hidden[-1,:,:]), dim=1)
        
        return encoder_hidden, encoder_outputs



class DecoderRNN(nn.Module):
    def __init__(self, output_size, embedding_size, num_encoders, num_decoders, hidden_size, bidirectional=False,dropout=0.0, cell='GRU'):
        super(DecoderRNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_encoders = num_encoders
        self.num_decoders = num_decoders
        self.bidirectional = bidirectional
        self.cell = cell
        self.embedding_size = embedding_size
        
        self.embedding = nn.Embedding(output_size, embedding_size)
        
        if self.cell == 'GRU':
            self.decoder_rnn = nn.GRU(embedding_size, hidden_size, num_layers=num_decoders,
                                      bidirectional=bidirectional, dropout =dropout)
        elif self.cell == 'LSTM':
            self.decoder_rnn = nn.LSTM(embedding_size, hidden_size, num_layers=num_decoders,
                                       bidirectional=bidirectional,dropout =dropout)
        else:
            self.decoder_rnn = nn.RNN(embedding_size , hidden_size, num_layers=num_decoders,
                                       bidirectional=bidirectional, dropout =dropout)
        
        
        
        self.decoder_fc = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=1)
    # def forward(self, input_seq, encoder_hidden, encoder_outputs):
    #     batch_size, max_len = input_seq.size()
        
        

    #     decoder_input = (input_seq.T)[0]

    #     print(decoder_input.shape)
    #     # Embedding layer
    #     embedded = self.embedding(decoder_input.long()).view(-1,batch_size, self.embedding_size)
        
        
       
    #     # Decoder layers
    #     decoder_hidden = encoder_hidden[:self.num_decoders]

    #     decoder_output = F.relu(embedded)

        

    #     decoder_output, decoder_hidden = self.decoder_rnn(embedded, decoder_hidden)
    #     decoder_output = self.softmax(self.decoder_fc(decoder_output))
    #     print(decoder_output.shape)
    #     # # Handle different numbers of layers in the encoder and decoder
    #     # if self.num_encoders != self.num_decoders:
    #     #     if self.num_encoders < self.num_decoders:
    #     #         # Copy all encoder hidden layers and then repeat the top layer
    #     #         top_layer_hidden = encoder_hidden[-1].unsqueeze(0)
    #     #         remaining_layers = self.num_decoders - self.num_encoders
    #     #         extra_hidden = top_layer_hidden.repeat(remaining_layers, 1, 1)
    #     #         decoder_hidden = torch.cat((encoder_hidden, extra_hidden), dim=0)
    #     #     else:
    #     #         # Slice the hidden states of the encoder to match the decoder layers
    #     #         decoder_hidden = encoder_hidden[-self.num_decoders:]
    #     # else:
    #     #     decoder_hidden = encoder_hidden
    #     #     decoder_outputs[:,i,:] = decoder_output.squeeze(1)
        
    #     # decoder_outputs = self.softmax(self.decoder_fc(decoder_outputs))
        
    #     return decoder_output

    def forward(self, input_seq, encoder_hidden, encoder_outputs):
        batch_size = input_seq.shape[0]
        
        size_of_hidden = encoder_hidden.shape[0]
        decoder_input = (input_seq)

        decoder_input = decoder_input.unsqueeze(1)
        
        # Embedding layer
        embedded = self.embedding(decoder_input.long()).view(-1,batch_size,self.embedding_size)

        decoder_hidden = encoder_hidden
        
        if(size_of_hidden>self.num_decoders):
          decoder_hidden = encoder_hidden[self.num_encoders-self.num_decoders:]
          
        elif(size_of_hidden<self.num_decoders):
          top = encoder_hidden[-1].unsqueeze(0)
          extra = self.num_decoders-self.num_encoders
          hiddenExtra =top.repeat(extra,1,1)
          decoder_hidden =torch.cat((encoder_hidden,hiddenExtra),dim=0)

        packed_output, decoder_hidden = self.decoder_rnn(embedded, decoder_hidden)
       
        # Project the output to the output size and apply softmax
        decoder_output = self.decoder_fc(packed_output)
        
        decoder_output = F.log_softmax(decoder_output, dim=2)
        
        # Return the output tensor
        return decoder_output,decoder_hidden

def calculate_accuracy(output_tensor, target_tensor):
    # Get the predicted word indices for each batch and sequence position
    predicted_word_indices = output_tensor.argmax(dim=2)
    
    # Compare with target tensor to get the number of correct predictions
    num_correct = torch.eq(predicted_word_indices, target_tensor).sum().item()
    
    # Calculate total number of predictions
    total_predictions = target_tensor.numel()
    
    # Calculate accuracy
    accuracy = num_correct / total_predictions
    
    return accuracy

In [101]:
# get encoded data of perticular language

data = EncodedData("mar")    

from torch.utils.data import TensorDataset

trainData = TensorDataset(data.train_xE, data.train_yE)




<class 'torch.Tensor'>


In [131]:

learn_rate = 0.0001
batch_size = 32
hidden_size = 32
embedding_size = 64
num_encoder = 3
num_decoder = 1
cell_type = 'RNN'
bidirectional = False
dropout = 0
nonlinearity = 'relu'
teach_ratio = 0.5
epochs = 10
input_size = data.input_lang.n_chars
output_size = data.output_lang.n_chars

encoder = EncoderRNN( input_size, embedding_size, num_encoder, hidden_size, bidirectional, cell_type,dropout)
decoder = DecoderRNN(output_size,embedding_size,num_encoder,num_decoder,hidden_size,bidirectional,dropout,cell_type)

trainLoader = DataLoader(trainData,batch_size=batch_size,shuffle=True) 

validData = TensorDataset(data.val_xE,data.val_yE)
validLoader = DataLoader(validData,batch_size=batch_size,shuffle=True) 

encoder_optimizer=optim.Adam(encoder.parameters(),learn_rate)
decoder_optimizer=optim.Adam(decoder.parameters(),learn_rate)
loss_fun=nn.CrossEntropyLoss(reduction="sum")

encoder.to(device)
decoder.to(device)

for i in range(epochs):
 
  running_loss = 0.0
  train_correct = 0

  encoder.train()
  decoder.train()

  for j,(train_x,train_y) in enumerate(trainLoader):
      train_x = train_x.to(device)
      train_y = train_y.to(device)

      encoder_optimizer.zero_grad()
      decoder_optimizer.zero_grad()

      

      

      
      
      encoder_hidden,encoder_output = encoder(train_x)
      
      # # lets move to the decoder
      

      # print(encoder_output.shape)
      # print(encoder_output.shape)
      # Compute the loss and accuracy
      
      decoder_input =(train_y.T)[0]
      decoder_hidden = encoder_hidden
      pred_output={}
      for k in range(train_y.shape[1]):

      
        decoder_output,decoder_hidden= decoder(decoder_input, encoder_hidden,encoder_output)
        encoder_hidden = decoder_hidden
        if(k==0):
          pred_output = decoder_output
        else:
          pred_output = torch.cat([pred_output,decoder_output],dim=0)
        

        # Get the index of the maximum probability in each row
        max_probs, indices = torch.max(decoder_output, dim=2)

# Reshape the indices tensor to (16,) for easier indexing
        indices = indices.view(batch_size)
      # compare the predicted output with the true labels
        decoder_input=indices

      
      loss = loss_fun(pred_output.view(-1, output_size), train_y.view(-1).long())
      running_loss += loss.item()
      # Compute the gradients and update the parameters
      loss.backward()
      encoder_optimizer.step()
      decoder_optimizer.step()

      # Compute the accuracy
      preds = torch.argmax(pred_output, dim=2).T
      train_correct += (preds == train_y).sum().item()
      # Print the training progress
      if j % 100 == 99:
          avg_loss = running_loss / (batch_size * 100)
          avg_acc = train_correct / (batch_size * 100 * train_y.size(1))
          print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.5f}%'
                .format(i+1, epochs, j+1, len(trainLoader), avg_loss, avg_acc*100))
          running_loss = 0.0
          train_correct = 0



Epoch [1/10], Step [100/1600], Loss: 86.7294, Accuracy: 19.68324%
Epoch [1/10], Step [200/1600], Loss: 69.4724, Accuracy: 50.66619%
Epoch [1/10], Step [300/1600], Loss: 59.3548, Accuracy: 51.10227%
Epoch [1/10], Step [400/1600], Loss: 55.8941, Accuracy: 51.14062%
Epoch [1/10], Step [500/1600], Loss: 54.3532, Accuracy: 51.28693%
Epoch [1/10], Step [600/1600], Loss: 53.7122, Accuracy: 51.05256%
Epoch [1/10], Step [700/1600], Loss: 53.0453, Accuracy: 51.18324%
Epoch [1/10], Step [800/1600], Loss: 52.8648, Accuracy: 51.02131%


KeyboardInterrupt: ignored

In [14]:
from torch.nn.modules.dropout import Dropout
input_size = data.input_lang.n_chars
output_size = data.output_lang.n_chars
EMBEDDING_SIZE=16
NUM_ENCODERS=2
NUM_DECODERS=2
HIDDEN_SIZE=32
BIDIRECTIONAL=False
BEAM_WIDTH = 1
DROPOUT=0.0
LEARNING_RATE=0.001
PAD_TOKEN=0
NUM_EPOCHS=1
batch_size=2